# Carga de datos de Airbnb desde CSV (en VSCode) a MySQL con SQLAlchemy
Este notebook:
1. Crea una base de datos MySQL llamada `airbnb_db`.
2. Crea tablas: `hosts`, `neighbourhoods`, `listings`, `reviews`.
3. Carga archivos CSV directamente a las tablas usando Pandas.

In [1]:
pip install sqlalchemy pymysql pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

usuario = "root"
contraseña = "123456789"
host = "localhost"
puerto = "3306"

# Crear motor inicial
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}")
conn = engine.connect()
conn.execute(text("DROP DATABASE IF EXISTS airbnb_db"))
conn.execute(text("CREATE DATABASE airbnb_db"))

# Conectarse a la nueva base
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/airbnb_db")
conn = engine.connect()

In [3]:
# Crear las tablas
conn.execute(text("""
CREATE TABLE hosts (
    host_id BIGINT PRIMARY KEY,
    host_name VARCHAR(255) NULL,
    calculated_host_listings_count INT NOT NULL
);
"""))

conn.execute(text("""
CREATE TABLE neighbourhoods (
    neighbourhood_id INT AUTO_INCREMENT PRIMARY KEY,
    neighbourhood_group VARCHAR(255) NULL,
    neighbourhood VARCHAR(255) NULL
);
"""))

conn.execute(text("""
CREATE TABLE listings (
    listing_id BIGINT PRIMARY KEY,
    name VARCHAR(255) NULL,
    host_id BIGINT NOT NULL,
    neighbourhood_id INT NOT NULL,
    room_type VARCHAR(255) NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    minimum_nights INT NOT NULL,
    availability_365 INT NOT NULL,
    FOREIGN KEY (host_id) REFERENCES hosts(host_id),
    FOREIGN KEY (neighbourhood_id) REFERENCES neighbourhoods(neighbourhood_id)
);
"""))

conn.execute(text("""
CREATE TABLE reviews (
    review_id INT AUTO_INCREMENT PRIMARY KEY,
    listing_id BIGINT NOT NULL,
    number_of_reviews INT NOT NULL,
    last_review DATE NULL,
    reviews_per_month DECIMAL(4,2) NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id)
);
"""))

In [ ]:
# Cargar CSVs
hosts_df = pd.read_csv("hosts.csv")
neighbourhoods_df = pd.read_csv("neighbourhoods.csv")
listings_df = pd.read_csv("listings.csv")
reviews_df = pd.read_csv("reviews.csv")

In [5]:
# Insertar en la base de datos
hosts_df.to_sql("hosts", con=engine, if_exists="append", index=False)
neighbourhoods_df.to_sql("neighbourhoods", con=engine, if_exists="append", index=False)
listings_df.to_sql("listings", con=engine, if_exists="append", index=False)
reviews_df.to_sql("reviews", con=engine, if_exists="append", index=False)

48895